## Data Science Preprocessing

Dieses Notebook ist einzig dafür vorgesehen, die Daten so weit vorzubereiten, dass sie im Notebook 3 visualisiert und analysiert werden können. Dies umfasst, die Datenbasis zu bereinigen und teilweise auch erste Berechnungsschritte. Auf dies wird jedoch einzeln bei den verschiedenen Zellen eingegangen.

In [1]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import locale
locale.setlocale(locale.LC_ALL, '')
from string import ascii_lowercase
import datetime as dt
from sklearn import linear_model
import time
import matplotlib.pyplot as plt
%matplotlib inline

### Preprocessing amerikanische Counties

Ziel dieser Zelle ist es die durchschnittlichen Wachstumsraten der Covid-Zahlen in amerikanischen Counties zu berechnen. Dafür wird die von der Non-Profit-Organisation USA Facts bereitgestellte Datenbasis in den ersten Zeilen bereinigt. Die Datenbasis sind die täglich gemeldeten Covid-Fälle jedes Counties. Da es in den Namen der Counties einzelne Duplikate gibt werden sie mit dem Staat, in dem das County sich befindet, in Zusammenhang gebracht. 
Danach wird ein Dictionary erstellt, in dem die Daten von jedem County in einem eigenen DataFrame abgespeichert werden. 
Außerdem wird das DataFrame avgwachstum erstellt, in dem die durchschnittlichen Wachstumsraten der Covid-Zahlen der Counties gespeichert werden, dafür wird durch jedes DataFrame des Dictionarys durchiteriert, da für jedes County die durchschnittlichen Werte bestimmt werden müssen.
Des Weiteren wird die Spalte Weekday hinzugefügt, bei der die Funktion day_name() genutzt wird, um den Wochentag einzufügen, um in der Analyse zu schauen, ob an bestimmten Wochentagen mehr Fälle gemeldet werden als an anderen.

In [2]:
counties = pd.read_csv('covid_confirmed_usafacts.csv')
counties["County Name"] = counties["County Name"] + "_" + counties["State"]

counties = counties.drop(['stateFIPS','countyFIPS',"State"], axis=1)

arraycounties = [counties['County Name'].tolist()]

dfState = pd.DataFrame()
dflistC = counties['County Name'].tolist()
dfState = counties[['County Name']]
dfState = dfState.reset_index()
dfState = dfState.drop_duplicates()
dflistC = list(dict.fromkeys(dflistC))
dic = {}

for i in dflistC:
    dic[i] = pd.DataFrame()
avgwachstum = pd.DataFrame(columns=["County","AGR"],index = range(len(dflistC)))

i = 0

for name, df in dic.items():
        dic[name] = counties.loc[(counties['County Name'] == name)]
        dic[name] = dic[name].reset_index()
        dic[name] = dic[name].T
        dic[name] = dic[name].rename(columns={0:'cases'})
        dic[name]['County Name'] = name
        dic[name] = dic[name].iloc[3:]
        dic[name] = dic[name].reset_index()
        dic[name] = dic[name].rename(columns={'index': 'date'})
        dic[name]['date'] = pd.to_datetime(dic[name]['date'], format='%m/%d/%y')
        dic[name] = dic[name].loc[dic[name]['date']< '2020-06-20']
        dic[name]['cases'] =  dic[name]['cases'].astype(int) 
        dic[name]['dailycases'] = dic[name]['cases'].diff()
        dic[name]['dailycases'][dic[name]['dailycases'] < 0] = 0
        dic[name]['Rate'] = dic[name]['dailycases'].pct_change()
        dic[name]['Rate'] = dic[name]['Rate']*100
        dic[name]['Rate'] = dic[name]['Rate'].replace([np.inf, -np.inf], 0)
        dic[name] = dic[name].fillna(0)
        dic[name]['Weekday'] = dic[name]['date'].dt.day_name()
        
        avgwachstum["County"][i] = name
        avgwachstum["AGR"][i] = dic[name]['Rate'].mean()#/len(dic[name].index)
        i +=1

In den nächsten zwei Zellen wird das DataFrame DFCounty erstellt. Dort werden die Daten der durchschnittlichen Wachstumsraten der Covid-Zahlen mit der Bevölkerungsdichte zusammengebracht. Da die Bevölkerungsdichte leider nicht gefunden werden konnte, musste sie selbst berechnet werden. Dafür werden die Bevölkerungszahlen der letzten Volkszählung (pop) und die Fläche der einzelnen Counties (areacounty) genommen. Diese müssen jeweils einzeln zugeordnet werden, da die jeweiligen Datenbasen verschieden viele Einträge haben, was sich darauf zurückführen lässt, dass zum Beispiel bei der Fläche auch noch die Staaten selbst als Summe aufgeführt sind.

In [3]:
areacounty = pd.read_excel('LND01.xls')
areacounty = areacounty[['Areaname','LND010190D']]
areacounty['Areaname'] = areacounty['Areaname'].str.replace(", "," County_", regex=True)

pop = pd.read_csv('covid_county_population_usafacts.csv')
pop["County Name"] = pop["County Name"] + "_" + pop["State"]

In [4]:
DFCounty = pd.DataFrame(avgwachstum)
DFCounty['Area'] = 0.0
DFCounty['Population'] = 0.0

for i in range(len(DFCounty)):
    DFCounty['Area'][i] = areacounty["LND010190D"].loc[areacounty["Areaname"] == DFCounty['County'][i]]
    DFCounty['Population'][i] = pop["population"].loc[pop["County Name"] == DFCounty['County'][i]]

DFCounty['Density'] = DFCounty['Population'] / DFCounty['Area'] 
DFCounty["State"] = DFCounty["County"].str[-2:]
DFCounty=DFCounty.replace([np.inf, -np.inf], np.nan)
DFCounty = DFCounty.fillna(0)
DFCounty = DFCounty[DFCounty['Density'] != 0]
DFCounty = DFCounty.fillna(0)

### Preprocessing deutsche Landkreise

In dieser Zelle wird das DataFrame kreise_zuwachs erstellt, bei dem der tägliche Zuwachs der Covid-Fälle in deutschen Landkreisen berechnet werden. Dafür werden bei der Datenbasis erst noch viele Spalten gelöscht und die Kreise alphabetisch sortiert.

In [5]:
kreise_zuwachs = pd.read_csv('COVID-19 Cases Germany_1604319250294.csv')
kreise_zuwachs.drop(['relative_case_changes','fb_id','cases_per_population','relative_death_changes','bundesland_ags','kreis_ags','kreis_nuts','kreis_name','deaths_per_population','fb_datetime'], axis=1, inplace=True)
kreise_zuwachs = kreise_zuwachs.fillna(0)
kreise_zuwachs['deaths'] = kreise_zuwachs['deaths'].astype(int)
kreise_zuwachs['publication_datetime'] = kreise_zuwachs['publication_datetime'].str[:11]
kreise_zuwachs = kreise_zuwachs.sort_values(['location_label','publication_datetime'])
kreise_zuwachs['täglicher Wachstum'] = kreise_zuwachs['cases'].pct_change()
kreise_zuwachs = kreise_zuwachs.drop(columns = ["population","source_label","source_website","bundesland_name"])

In dieser Zelle gibt es das gleiche Vorgehen wie in Zeile [2], also wie bei den Counties nur für deutsche Landkreise. Jedoch kommen hier noch die durchschnittlichen Wachstumsraten der Covid-Tode der Landkreise hinzu. Die jeweiligen täglichen Zahlen der Landkreise werden in dem Dictionary d gespeichert.

In [6]:
arraykreise = kreise_zuwachs['location_label'].tolist()
arraykreise = list(dict.fromkeys(arraykreise))  #Entfernt alle Dubletten
arraykreise = sorted(arraykreise)
dflist = arraykreise
d = {} #dict damit können alle erstelleten DataFrames aufgerufen werden

for j in dflist:
   d[j] = pd.DataFrame()
i = 0


durchschnittlicheswachstum = pd.DataFrame(columns=["Landkreis","AGR"],index = range(len(arraykreise)))
durchschnittlicheswachstumD = pd.DataFrame(columns=["Landkreis","AGR"],index = range(len(arraykreise)))

for name, df in d.items():
        d[name] = kreise_zuwachs.loc[kreise_zuwachs['location_label'] == name]
        d[name]['dailycases'] = d[name]['cases'].diff()
        d[name]['dailydeaths'] = d[name]['deaths'].diff()
        d[name]['dailycases_per_100k'] = d[name]['cases_per_100k'].diff()
        d[name]['dailydeaths_per_100k'] = d[name]['deaths_per_100k'].diff()
        d[name]['publication_datetime'] = d[name]['publication_datetime'].astype('datetime64[ns]')
        d[name] = d[name].sort_values(['publication_datetime'])
        d[name] = d[name].loc[d[name]['publication_datetime']< '2020-06-20']
        d[name]['dailycases'][d[name]['dailycases'] < 0] = 0
        d[name]['Rate'] = d[name]['dailycases'].pct_change()
        d[name]['Rate'] = d[name]['Rate']*100
        d[name]['Rate'] = d[name]['Rate'].replace([np.inf, -np.inf], 0)
        
        d[name]['RateD'] = d[name]['dailydeaths'].pct_change()
        d[name]['RateD'] = d[name]['RateD']*100
        d[name]['RateD'] = d[name]['RateD'].replace([np.inf, -np.inf], 0)
        d[name] = d[name].fillna(0)
        d[name] = d[name].reset_index()
        
        if d[name].empty:
            time.sleep(1)
        else:
            d[name]['Wochentag'] = d[name]['publication_datetime'].dt.day_name()
        
        durchschnittlicheswachstum["Landkreis"][i] = name
        durchschnittlicheswachstum["AGR"][i] = d[name]['Rate'].sum()/len(d[name].index)
        
        durchschnittlicheswachstumD["Landkreis"][i] = name
        durchschnittlicheswachstumD["AGR"][i] = d[name]['RateD'].sum()/len(d[name].index)
        i = i + 1

In dieser Zelle wird die Bevölkerungsdichte deutscher Landkreise den durchschnittlichen Wachstumsraten der Covid-Fälle zugeordnet. Problematisch hierbei ist, dass es zu Duplikaten kommt, da zum Beispiel Städte wie Würzburg, die auch Landkreise sind, doppelt aufgeführt sind. Dafür wird die erste For-Schleife genutzt, um das Wort Stadt an den "Landkreis" mit der höheren Bevölkerungsdichte gehangen, da dies die Stadt ist und nicht ein ländlicher Landkreis.

In [7]:
dichte = pd.read_excel('A_BevDichte_Kreise.xlsx')
dichte = dichte.drop(columns = ['Kennziffer','Thünen-Typ'])
dichte = dichte.sort_values(by=['Name'])
dichte = dichte.reset_index()
dichte = dichte.drop(columns = 'index')

#Wird genutzt um die Stadt von dem Landkreis zu trennen
name = ' Stadt'

for j in range(len(dichte)):
    if j == 401:
        break
    else:
        if dichte['Name'][j] == dichte['Name'][j+1]:
            if dichte['Bevölkerungsdichte 2014'][j] < dichte['Bevölkerungsdichte 2014'][j+1]:
                dichte['Name'][j+1] = dichte['Name'][j+1] + name
            else:
                dichte['Name'][j] = dichte['Name'][j] + name

dichteU = pd.DataFrame(columns= ['Landkreis/Kreis','Bev.D.Ew./km²'])
dichteU['Landkreis/Kreis'] = arraykreise
dichteU = dichteU.fillna(0)

durchschnittlicheswachstum["Bev.D.Ew./km²"] = 0.0

for i in range(len(arraykreise)):
    durchschnittlicheswachstum['Bev.D.Ew./km²'][i] = dichte['Bevölkerungsdichte 2014'].loc[dichte["Name"] == dichteU['Landkreis/Kreis'][i]]

Zuletzt wird das DataFrame compLKfnl erstellt, in dem nun alle Werte, die durchschnittliche Wachstumsrate der Fallzahlen und der Todeszahlen mit der Bevölkerungsdichte zusammengebracht werden. Außerdem werden alle nan Werte, falls es welche geben sollte, durch 0 ausgetauscht. Des Weiteren muss die Spalte mit den Landkreisnamen gelöscht werden, da ansonsten das Clustering in Notebook 3 nicht funktioniert.

In [8]:
compLK = pd.DataFrame(durchschnittlicheswachstum)
compLK['AGR_D'] = durchschnittlicheswachstumD["AGR"]
compLK['Bev.D.Ew./km²'] = compLK['Bev.D.Ew./km²'].astype(float)
compLK = compLK.fillna(0)
compLK = compLK.loc[compLK['Bev.D.Ew./km²'] != 0.0]
compLK = compLK.loc[compLK['AGR'] != 0.0]
compLK = compLK.loc[compLK['AGR_D'] != 0.0]
compLKfnl = compLK.drop(columns =['Landkreis'])
compLKfnl = compLKfnl.rename(columns={'Bev.D.Ew./km²':'Dichte'})
compLKfnl = compLKfnl.fillna(0)

### Preprocessing deutsche Bundesländer

In der nächsten Zelle werden die täglichen Covid-Fallzahlen der einzelnen Bundesländer vorbereitet. So zum Beispiel werden in der Datenbasis, sollte es noch keine Fallzahlen geben der Wert als "—" dargestellt, für Berechnungen wird dies durch eine 0 ausgetauscht. Des Weiteren werden Tausender durch einen Punkt abgegrenzt und es haben sich teilweise Klammern eingeschlichen. Ziel ist es, dass die Spalte in den Typen int umgeändert werden kann.  
Außerdem werden die Daten in ein Trainings- und ein Test-Set aufgeteilt. Sodass das Trainings-Set aus Februar, März, April, Mai und Juni bis zum 19.06 besteht. 

In [9]:
bundzuwachsraw = pd.read_csv('table-1.csv')
bundzuwachsraw.set_index('Datum')
bundzuwachsraw = bundzuwachsraw.drop(['KW','Differenz zum Vortag','Bem.','Datum'],1) 
bundzuwachsraw = bundzuwachsraw.iloc[:-1]
bundzuwachsraw = bundzuwachsraw.fillna(0)
bundzuwachsraw = bundzuwachsraw.replace('—',0)

for column in bundzuwachsraw:
    bundzuwachsraw[column] = bundzuwachsraw[column].astype(str)
    bundzuwachsraw[column] = bundzuwachsraw[column].str.replace('.','')
    for c in ascii_lowercase:
        bundzuwachsraw[column] = bundzuwachsraw[column].str.replace(c,'')
    bundzuwachsraw[column] = bundzuwachsraw[column].str.replace(')','')
    bundzuwachsraw[column] = bundzuwachsraw[column].str.replace('(','')
    bundzuwachsraw[column] = bundzuwachsraw[column].astype(int)
    bundzuwachsraw[column] = bundzuwachsraw[column].diff()

bundzuwachsraw = bundzuwachsraw.fillna(0)
bundzuwachsraw['Date'] = pd.date_range(start='24/2/2020', periods=len(bundzuwachsraw), freq='D')        
bundzuwachsraw = bundzuwachsraw.set_index('Date')

bundzuwachsraw_1 = bundzuwachsraw.iloc[:118] # Februar Tage + März + April + Mai + Juni bis zum 19
bundzuwachsraw_2 = bundzuwachsraw.iloc[118:]

Nun wird das Datum in einen Integer Wert mit dt.datetime.toordinal umgewandelt, um in der nächsten Zelle die lineare Regression anzuwenden. 

In [10]:
bundzuwachsraw_1['date'] = bundzuwachsraw_1.index
bundzuwachsraw_1['datef'] = pd.to_datetime(bundzuwachsraw_1['date'])
bundzuwachsraw_1['datef']=bundzuwachsraw_1['datef'].map(dt.datetime.toordinal)
bundzuwachsraw_2['date'] = bundzuwachsraw_2.index
bundzuwachsraw_2['datef'] = pd.to_datetime(bundzuwachsraw_2['date'])
bundzuwachsraw_2['datef']=bundzuwachsraw_2['datef'].map(dt.datetime.toordinal)

In dieser Zelle , werden die vorbereiteten Trainingsdaten der Bundesländer in ein Model der linearen Regression gefittet. Und dann für den Testzeitraum Prognosen erstellt. 
Nachdem für jedes Bundesland Prognosen erstellt worden sind, werden nun die durchschnittlichen täglichen Wachstumsraten der Fallzahlen für jedes Bundesland berechnet und in dem DataFrame durchwachstumfnl gespeichert.

In [11]:
bundzuwachspred = pd.DataFrame(bundzuwachsraw_2['date'])

for column in bundzuwachsraw_1:
    y = bundzuwachsraw_1[column].to_numpy()
    x = bundzuwachsraw_1['datef'].to_numpy()
    xpred = bundzuwachsraw_2['datef'].to_numpy()
    ypred = bundzuwachsraw_2[column].to_numpy()
    x = x.reshape(-1, 1)
    y = y.reshape(-1, 1)
    xpred = xpred.reshape(-1, 1)
    regr = linear_model.LinearRegression()
    regr.fit(x, y)
    predic = regr.predict(xpred)
    bundzuwachspred[column] = predic
bundzuwachspred = bundzuwachspred.drop(columns = ['date','datef','Gesamt'])
bundchangepred = bundzuwachspred.pct_change()
bundchangepred = bundchangepred*100
bundchangepred = bundchangepred.fillna(0)
bundchangepred = bundchangepred.replace([np.inf, -np.inf], 0)
bundchangepred = bundchangepred.drop(bundchangepred.index[0])

durchschnittlichwachstum = bundchangepred.mean()
durchwachstumfnl = pd.DataFrame({'Bundesland':durchschnittlichwachstum.index,'AGR':durchschnittlichwachstum.values})
liste = ['Baden-Württemberg','Bayern','Berlin','Brandenburg','Bremen','Hamburg','Hessen','Mecklenburg-Vorpommern','Niedersachsen','Nordrhein-Westfalen','Rheinland-Pfalz','Saarland','Sachsen','Sachsen-Anhalt','Schleswig-Holstein','Thüringen']

for i, row in durchwachstumfnl.iterrows():
        durchwachstumfnl.at[i,'Bundesland'] = liste[i]